In [125]:
from accounts import si

#si.ID, si.PW

In [ ]:
!pip install BeautifulSoup4
!pip install lxml
!pip install selenium
!pip install selenium --upgrade
!pip install webdriver-manager

In [216]:
os.getcwd()

'c:\\Users\\Playdata\\Desktop\\github\\MiniProj\\modules\\crawlers'

In [106]:
import os
from datetime import datetime
import time
import requests
import re

import pandas as pd

from bs4 import BeautifulSoup

import selenium


from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [ ]:
'''
class SRICrawler():
사람인 -> url ->pc status code 200
0.셀레늄이용 먼저 로그인 엔터 누르기 
1.검색창에 입력값 (회사이름) 입력 시 
->회사 리뷰까지 접속 
->코멘트를 뭐였지 아무튼 검사로 체크해서 들어가기
->
'''

In [222]:
service = Service(executable_path='chromedriver.exe')
browser = webdriver.Chrome(service=service)
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'

In [223]:
url = r"https://www.saramin.co.kr/zf_user/"
save_path=f'C:\python_lesson\mini_project\MiniProj\si_crawl\키워드_si.csv'

browser.maximize_window()
print(browser.get_window_size())

browser.get(url)

{'width': 1552, 'height': 832}


In [224]:

#로그인 버튼 클릭
login_screen_btn = browser.find_element(By.CSS_SELECTOR,"#sri_header > div.wrap_header > div.utility > div.sign > a.btn_sign.signin") 
login_screen_btn.click()
#아이디 버튼 클릭
id_btn = browser.find_element(By.CSS_SELECTOR,"#id")
#id_btn.click()
#아이디 입력
id_btn.send_keys(si.ID)

#비밀번호 버튼 클릭
pwd_btn = browser.find_element(By.CSS_SELECTOR,"#password")
pwd_btn.click()
#비밀번호 입력
pwd_btn.send_keys(si.PW)

#로그인 클릭
login_btn = browser.find_element(By.CSS_SELECTOR,"#login_frm > div > div > div.login-form > button").send_keys(Keys.ENTER)

In [225]:
#메인 홈페이지로 돌아가기
main_btn = browser.find_element(By.CSS_SELECTOR,'#sri_gnb_wrap > a > svg.img_ci')
main_btn.click()


In [226]:
category_btn = browser.find_element(By.CSS_SELECTOR,'#sri_header > div.wrap_header > div.navigation > ul > div:nth-child(4) > a > span')
category_btn.click()


In [228]:
#검색창 클릭
try: #처음 들어갈 때 팝업창이 떴을 경우
    pop_up_delete_btn= browser.find_element(By.CSS_SELECTOR,'#wrap_review_tutorial > div > button')
    pop_up_delete_btn.click()
    search_btn = browser.find_element(By.CSS_SELECTOR,'#content > div.company_review_main > div.wrap_main_top > div > div > div.area_input > input')
    search_btn.click()
    
except:    
    search_btn = browser.find_element(By.CSS_SELECTOR,'#content > div.company_review_main > div.wrap_main_top > div > div > div.area_input > input')
    search_btn.click()


In [229]:
#검색어 입력
cop=input()
print(cop)
search_btn.send_keys(cop)


삼성전자


In [230]:
#검색어 엔터
enter_btn = browser.find_element(By.CSS_SELECTOR,'#content > div.company_review_main > div.wrap_main_top > div > div > div.area_input.on > button > svg')
enter_btn.click()


In [231]:
#(주) 앞뒤로 포함돼 있고 대상 검색 문자열(cop)과 일치
i=1
while True:
    try: #(주)가 붙은회사
        n_btn = browser.find_element(By.CSS_SELECTOR, f" div:nth-child({i}) > div > div.area_info > strong > a:nth-child(1)") #뒤에 (주)글자까지 포함
        
        
        if cop+'(주)' == n_btn.text or '(주)'+cop == n_btn.text or cop == n_btn.text: #주 붙은 회사 먼저 while문 크롤링   
            n_btn.click() #주가 붙어있는 회사라면 클릭   
            break       
        i += 1
                
            
    except: # (주)가 붙지 않은 회사
        print(f"No result found for {cop}") #주 붙지 않은 회사 찾기 위해 초기화
        break


In [232]:
#한번 크롤링하고 클릭하고 리스트에 넣고 그걸 끝날 때까지 반복하고 break
review_list=[]
review_tag=[]#div.bx-wrapper > div > ul > li:nth-child({i}) > p.desc
i=0
while True:
        try:
                i+=1 #하나씩 리뷰 넘어가기 위함

                if len(browser.window_handles) >= 2: #브라우저 창이 여러개라면 
                # 새로 띄워진 창으로 전환
                        browser.switch_to.window(browser.window_handles[-1]) # 새로 띄워진 창이 맨 마지막 창이라 가정

                        # 새로 띄워진 창의 URL 가져오기
                        new_window_url = browser.current_url

                html=browser.page_source
                soup = BeautifulSoup(html, "html.parser")# html 파싱 
                review_tag= soup.select(f"div.bx-wrapper > div > ul > li:nth-child({i}) > p.desc") #파싱된 html 내용을 담아주기
                review_list.append(review_tag[0].text.strip()) #담긴 리뷰 리스트를 strip해서 review_list에 담아주고
                
        except: # 아예 리뷰가 끝나버리면 break 
                break



In [233]:
print(len(review_list)) #리뷰 리스트에 잘 들어왔는지 확인

45


In [204]:
#오후 일정 만든 리뷰 리스트를 토대로 데이터프레임 생성후 데이터 전처리 및 예외처리할 부분 하기

In [234]:
#데이터프레임 이름은 키워드_daum.csv
si_df=pd.DataFrame(review_list,columns=[f'review'])
si_df


,review
0,대표 성장 기업으로서 업계 리딩 차원에서 제반\n업무들이 경력 개발에 도움이 되고 ...
1,글로벌회사에 걸맞는 회사로 복지 및 다양한 혜택이 늘어나고 직원들에 대한\n전폭적인...
2,부서에 따라 차이가 나지만 일부 부서를 제외하고 업무량이 많지는 않다. 성과금이 전...
3,글로벌 회사에서 좋은 시스템을 기반으로 뛰어난 동료들과 일할 수 있음
4,"높은 성과급, 자율출퇴근"
5,부서따라 다르지만 워라벨 좋음
6,복리후생이 좋은 대기업
7,네임밸류가 있어 이직시 도움이 된다
8,최고의 회사 함께 성장하는 회사 및 자기개발 지원 많이 해주는 좋은 회사 입니다
9,안정적입니다\n급여가 높고 복지가 잘 되어있어요


In [238]:
si_df['review'] = si_df['review'].apply(lambda x: re.sub(r'\s+', ' ',re.sub(r'\n+', ' ', x)).strip())

In [240]:
si_df

,review
0,대표 성장 기업으로서 업계 리딩 차원에서 제반 업무들이 경력 개발에 도움이 되고 자...
1,글로벌회사에 걸맞는 회사로 복지 및 다양한 혜택이 늘어나고 직원들에 대한 전폭적인 ...
2,부서에 따라 차이가 나지만 일부 부서를 제외하고 업무량이 많지는 않다. 성과금이 전...
3,글로벌 회사에서 좋은 시스템을 기반으로 뛰어난 동료들과 일할 수 있음
4,"높은 성과급, 자율출퇴근"
5,부서따라 다르지만 워라벨 좋음
6,복리후생이 좋은 대기업
7,네임밸류가 있어 이직시 도움이 된다
8,최고의 회사 함께 성장하는 회사 및 자기개발 지원 많이 해주는 좋은 회사 입니다
9,안정적입니다 급여가 높고 복지가 잘 되어있어요


In [239]:

# 절대경로 지정 및 저장할 파일 이름

# 데이터프레임을 CSV 파일로 저장
df.to_csv(save_path, index= True, encoding='utf-8-sig')

In [ ]:
# 엔터랑 양 끝 공백제거 